# 201910812 안윤호

In [118]:
%%writefile ../src/CusOrder.sol
pragma solidity ^0.6.0;

contract Customer {
    uint id;
    string name;
    string ph;
    string home;
    
    mapping(address => uint256) public balanceOf;
    mapping(string => address) playersAddr;
    
    function addCustomer(uint _id, string memory _name, string memory _ph, string memory _home)  public payable {
        id=_id;
        name=_name;
        ph=_ph;
        home=_home;
    }
    
    function getHomeAddress() public view returns(string memory) {
        return home;
    }
    
    function getId() view public returns(uint) {
        return id;
    }
}


contract Order {
    
    mapping(uint=>Customer) customerById;
    
    struct OrderStruct{
        uint id;
        string p;
        uint n;
        uint amount;
        
        uint time;
        string status;
        string shipping;
    }
    uint orderLen = 0;
    uint totalAmount = 0;
    
    mapping(address=>OrderStruct) orderMap;
    mapping(uint=>address) addressById;
    mapping(address=>string) homeByAddr;
    
    address payable owner;
    constructor() public {
        owner = msg.sender;
    }
    
    
    function placeOrder(uint _id, string memory _p, uint _n, uint _amount) public payable{
        
        string memory  home = getHomeAddress();
        OrderStruct memory o=OrderStruct(_id, _p, _n, _amount, now, "Orderd", home);
        
        orderMap[tx.origin] = o;
        addressById[_id] = tx.origin;
        orderLen += 1;
        totalAmount += _amount;
    }
    
    function addCustomer(uint _id, string memory _name, string memory _ph, string memory _home) public payable{
        Customer cus = new Customer();
        cus.addCustomer(_id, _name, _ph, _home);
        
        customerById[_id]=cus;
        homeByAddr[tx.origin] = _home;
    }
    

    function getHomeAddress() public view returns(string memory){
        string memory home = homeByAddr[tx.origin];
        return home;
    }
    
    function getStatus() public view returns(string memory){
        return orderMap[tx.origin].status;
    }
    
    function updateStatus(uint _id, string memory _s) public isOwner {
        orderMap[addressById[_id]].status = _s;
    }
    
    function getOrderItem() public view returns(uint, string memory, string memory, string memory){
        return (orderMap[tx.origin].id, orderMap[tx.origin].p, orderMap[tx.origin].status, orderMap[tx.origin].shipping);
    }
    
    function getOrderById(uint _id)  public view returns(uint, string memory, string memory, string memory){
        OrderStruct memory o = orderMap[addressById[_id]];
        return (o.id, o.p, o.status, o.shipping);
    }
    
    function getNOrder() public view returns(uint){
        return orderLen;
    }
    
    function getTotalOrderAmount() public view returns(uint){
        return totalAmount;
    }
    
    function queryBalance()  view public returns(uint){
        return address(this).balance;
    }
    
    modifier isOwner{
        require(msg.sender == owner);
        _;
    }

}

Overwriting ../src/CusOrder.sol


In [119]:
!solc ../src/CusOrder.sol --combined-json abi > ..\src\CusOrderABI.json

In [120]:
!solc ../src/CusOrder.sol --combined-json bin > ..\src\CusOrderBIN.json

## 배포

In [121]:
%%writefile ../src/CusOrderDeploy.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');

var web3;
web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));


contractName=Object.keys(_abiJson.contracts); 
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);
_bin=_binJson.contracts[contractName[1]].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();

    //gas check
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1662136, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/CusOrderDeploy.js


In [122]:
!node ../src/CusOrderDeploy.js

>> gas: 1660432
hash: 0x17f87f48b85b5712bd607d76d2f479ed2f7356a61263261bf742ebaf2a4f6721
---> The contract deployed to: 0x3DAD82caa77A687fc05AefD924056924B0905Eda


## 사용

# 문제3의 답

In [76]:
%%writefile ../src/CusOrderUse.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
_bin=_binJson.contracts[contractName[1]].bin;
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);

var _order = new web3.eth.Contract(_abiArray, '0x7F1a224C26BA3856442300Edd79D8236bAB6dd10');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balance0 = await web3.eth.getBalance(accounts[0]);
    const balance1 = await web3.eth.getBalance(accounts[1]);
    const balance2 = await web3.eth.getBalance(accounts[2]);
    
    console.log(`account0: ${accounts[0]}, balance: ${balance0}`)
    console.log(`account1: ${accounts[0]}, balance: ${balance1}`)
    console.log(`account2: ${accounts[0]}, balance: ${balance2}\n`)
    
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log("gas: "+ gas);
    });
    
    // wait and exit
    setTimeout(function() {
      process.exit(1);
    }, 1000);
}

doIt()

Overwriting ../src/CusOrderUse.js


In [77]:
!node ../src/CusOrderUse.js

account0: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000
account1: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000
account2: 0x1CEc51b65ef4A8b0124581587FC9F846FC2DAF83, balance: 100000000000000000000

gas: 1662136


# 문제4의 답

In [127]:
%%writefile ../src/CusOrderUse.js
var Web3=require('web3');
var _abiJson = require('./CusOrderABI.json');
var _binJson = require('./CusOrderBIN.json');


var web3;
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); 
_bin=_binJson.contracts[contractName[1]].bin;
_abiArray=JSON.parse(_abiJson.contracts[contractName[1]].abi);

var _order = new web3.eth.Contract(_abiArray, '0x3DAD82caa77A687fc05AefD924056924B0905Eda');

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    
    
    await _order.methods.addCustomer(111, "kim", "010-2017-1111", "111 hongji-dong jongro-gu seoul")
    .send({from: accounts[0], gas: 300000})
    
    await _order.methods.addCustomer(112, "lee", "010-2017-1112", "112 hongji-dong jongro-gu seoul")
    .send({from: accounts[1], gas: 300000})
    
    await _order.methods.addCustomer(113, "lim", "010-2017-1113", "113 hongji-dong jongro-gu seoul")
    .send({from: accounts[2], gas: 300000})
    
    var home = await _order.methods.getHomeAddress.call({from: accounts[1]})
    console.log(home)
    
    // wait and exit
    setTimeout(function() {
      process.exit(1);
    }, 5000);
}

doIt()

Overwriting ../src/CusOrderUse.js


# 문제5 풀지 못함

In [128]:
!node ../src/CusOrderUse.js

{
  call: [Function: bound _executeMethod] {
    request: [Function: bound _executeMethod]
  },
  send: [Function: bound _executeMethod] {
    request: [Function: bound _executeMethod]
  },
  encodeABI: [Function: bound _encodeMethodABI],
  estimateGas: [Function: bound _executeMethod],
  arguments: [],
  _method: {
    inputs: [],
    name: 'getHomeAddress',
    outputs: [ [Object] ],
    stateMutability: 'view',
    type: 'function',
    constant: true,
    payable: undefined,
    signature: '0x478e8461'
  },
  _parent: Contract {
    currentProvider: [Getter/Setter],
    _requestManager: RequestManager {
      provider: [WebsocketProvider],
      providers: [Object],
      subscriptions: {}
    },
    givenProvider: null,
    providers: {
      WebsocketProvider: [Function: WebsocketProvider],
      HttpProvider: [Function: HttpProvider],
      IpcProvider: [Function: IpcProvider]
    },
    _provider: WebsocketProvider {
      responseCallbacks: {},
      notificationCallbacks: [Ar